In [1]:
# Importing the required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import datetime
import numpy as np
import sqlite3


In [2]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Locate the table based on the given attributes
    table = soup.find('table', table_attribs)
    
    if not table:
        raise ValueError("Table not found with the given attributes")
    
    # Read the table into a DataFrame
    df = pd.read_html(str(table))[0]
    
    
    # Clean the "Market Cap" column if it exists
    if 'MC_USD_Billion' in df.columns:
        df['MC_USD_Billion'] = df['MC_USD_Billion'].str.replace('\n', '', regex=False).astype(float)

    
    return df


In [3]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    # Read the CSV file into a DataFrame
    exchange_rates = pd.read_csv(csv_path)
    
    # Convert the DataFrame into a dictionary
    exchange_rate_dict = exchange_rates.set_index('Currency').to_dict()['Rate']
    
    # Add new columns for different currencies
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate_dict['GBP'], 2) for x in df['Market cap (US$ billion)']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate_dict['EUR'], 2) for x in df['Market cap (US$ billion)']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate_dict['INR'], 2) for x in df['Market cap (US$ billion)']]
    
    return df


In [4]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    # Load DataFrame to SQL table
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    

In [5]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    try:
        # Print the query statement
        print(f"Executing query: {query_statement}")
        
        # Execute the query and fetch results into a DataFrame
        df = pd.read_sql_query(query_statement, sql_connection)
        
        # Print the query output
        print("Query output:")
        print(df)
        
        return df

    except Exception as e:
        # Log and print any errors
        print(f"An error occurred: {e}")
        

In [6]:
# Extract data and log progress
url ='https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = {'class': 'wikitable'}
df = extract(url, table_attribs)
print(df)

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81


In [7]:

# Path to the exchange rate CSV file
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

# Apply the transform function
df_transformed = transform(df, csv_path)

# Print the DataFrame to check the results
print(df_transformed)

output_path = './Largest_banks_data.csv'
load_to_csv(df_transformed, output_path)
conn = sqlite3.connect('Banks.db')
table_name = 'Largest_banks'
load_to_db(df_transformed, conn, table_name)

result_df = run_query('SELECT * FROM Largest_banks', conn)
result_df = run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)
result_df = run_query('SELECT [Bank name] from Largest_banks LIMIT 5', conn)

conn.close()


   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31    

In [8]:
df_loaded = pd.read_csv('./Largest_banks_data.csv')
df_loaded.head()

,Rank,Bank name,Market cap (US$ billion),MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,2,Bank of America,231.52,185.22,215.31,19204.58
2,3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,5,HDFC Bank,157.91,126.33,146.86,13098.63
